
```
                             Alexey Kuleshevich
                             
                        HaskellerZ - 28th January 2021
                        
                            New random interface
                             
                             
                             
```

## History

### Ancient

* At some point in the 90s `Random` module came into being and was included with ghc for
  many years until about a decade ago. Latest known versions that were wired in
  with GHC:
  * `random-1.1` briefly in `ghc-8.4.2` and `ghc-8.2.2`
  * before that `<= random-1.0.0.3` in `ghc-7.0.4` and older.

* February 1999 - Original version made it into [Haskell 98
  report](https://www.haskell.org/onlinereport/random.html) and is included in
  [`haskell98`](https://hackage.haskell.org/package/haskell98) package.

* July 2005 - first ticket on ghc tracker
  [#427](https://gitlab.haskell.org/ghc/ghc/-/issues/427) about `Random.StdGen` being
  slow, which hasn't been fixed until now.

* November of 2007 -
  [`random-1.0.0.0`](https://hackage.haskell.org/package/random-1.0.0.0) is available on
  Hackage


### Recent

* December 2019 - I wrote a blog post about how terrible `random`'s performance is:
  [Random benchmarks](https://alexey.kuleshevi.ch/blog/2019/12/21/random-benchmarks/).

* February 2020 - Dominic Steinitz confirmed my benchmark results from the blogpost and 
  asked if I'd be willing to collaborate on getting those issues with `random` resolved.

* February 2020 - June 2020: Dominic Stenitz, Leonhard Markert and myself worked really
  hard on improving `random` by solving it's most serious issues.

* 23rd of June of 2020 - all the hard work paid off and culminated in
  [random-1.2.0](https://hackage.haskell.org/package/random-1.2.0) being released.

## Original interface

A type class for Pseudo Random Number Generator (PRNG) implementers:

```haskell
class RandomGen g where
  genRange :: g -> (Int, Int)
  next     :: g -> (Int, g)
  split    :: g -> (g, g)
```

A type class for values that can be generated using any pure PRNG that provides `RandomGen`
instance:

```haskell
class Random a where
  randomR :: RandomGen g => (a, a) -> g -> (a, g)
  random  :: RandomGen g => g -> (a, g)

  randomRs :: RandomGen g => (a, a) -> g -> [a]
  randoms  :: RandomGen g => g -> [a]

  randomRIO :: (a, a) -> IO a
  randomIO  :: IO a
```

### Using original interface
#### Sample data type

In [1]:
{-# LANGUAGE GeneralizedNewtypeDeriving #-}
{-# LANGUAGE NamedFieldPuns #-}
import Text.Printf

newtype AreaCode = AreaCode { unAreaCode :: Int }
  deriving (Eq, Show, Num)

-- | The North American Numbering Plan (NANP) phone, eg. +1-555-123-4567
data Phone = Phone { phoneAreaCode :: AreaCode
                   , phoneLocalNumber :: Int
                   }

instance Show Phone where
  show Phone {phoneAreaCode, phoneLocalNumber} =
    let areaCode = unAreaCode phoneAreaCode
        (phoneSuffix, phonePostfix) = phoneLocalNumber `quotRem` 10000
     in printf "+1-%03d-%03d-%04d" areaCode phoneSuffix phonePostfix

#### Generating random data

In [2]:
import System.Random

randomPhone :: RandomGen g => [AreaCode] -> g -> (Phone, g)
randomPhone areaCodes g =
  let (i, g') = randomR (0, length areaCodes - 1) g
      (phoneLocalNumber, g'') = randomR (0, 9999999) g'
   in (Phone {phoneAreaCode = areaCodes !! i, phoneLocalNumber}, g'')

In [26]:
gen <- getStdGen
let (tollFreePhone, gen') = randomPhone [800,833,844,855,866,877,888] gen
tollFreePhone

+1-833-828-9549

In [4]:
let (newMexicoPhone, gen'') = randomPhone [505,575] gen'
newMexicoPhone

+1-575-676-4875

#### Splitting pure generator

In [5]:
let (gen1, gen2) = split gen''
-- Run in parallel in separate threads or even computers
randomPhone [800,833,844,855,866,877,888] gen1
randomPhone [800,833,844,855,866,877,888] gen2

(+1-844-539-4794,StdGen {unStdGen = SMGen 6002505185881207226 15484608899572013139})

(+1-844-530-8651,StdGen {unStdGen = SMGen 15179110519218422382 751779991543338727})

## Issues that we wanted to solve

* Quality of default pseudo-random number generator (PRNG)

* Terrible performance

* Interface is not 100% correct

* Design an interface that can also work with stateful generators

## Problem
### StdGen

Original PRNG implementation:

* Bad quality of randomness

* Generated only ~31bits of data at a time (`[1, 2147483562]` range)

* Bad performance characteristics

* Splitting produced sequences that weren't independent


### StdGen (Solution)

Switched to [`splitmix`](https://hackage.haskell.org/package/splitmix)
package for `StdGen` implementation:

* Very good quality of randomness (passes most tests, eg. Dieharder)

* Fastest PRNG implementation in Haskell

* Generates 64bits of random data in one iteration

* Splitting produces independent sequences

## Problem
### Poor performance

* Previous implementation of `StdGen` was slow.

* Generation of **all** types went through `Integer`.

* `genRange` was a historical mistake that was necessary for some PRNGs that produced values
  in unusual ranges, like the original StdGen

### Performance (Solutions)

* Switched `StdGen` to `splitmix`, as was already mentioned earlier.

* Used [bitmask with rejection technique](https://www.pcg-random.org/posts/bounded-rands.html)
  for generating values in custom ranges.

* Major redesign of `RandomGen` class:

  * Deprecated `genRange` and `next` in favor of `genWord64` and/or `genWord32`

  * Made generation of other bit widths customizable: `genWord[8|16|32|64]` and
    `genWord[32|64]R`.

  * Allowed customization of array generation with `genShortByteString`. More on this later.

### Performance (Solution)

New definition of `RandomGen` class (default implementations are omitted):

```haskell
class RandomGen g where
  {-# MINIMAL split,(genWord32|genWord64|(next,genRange)) #-}
  genWord8 :: g -> (Word8, g)
  genWord16 :: g -> (Word16, g)
  genWord32 :: g -> (Word32, g)
  genWord64 :: g -> (Word64, g)
  genWord32R :: Word32 -> g -> (Word32, g)
  genWord64R :: Word64 -> g -> (Word64, g)
  genShortByteString :: Int -> g -> (ShortByteString, g)
  split :: g -> (g, g)

  -- Deprecated:
  genRange :: g -> (Int, Int)
  next :: g -> (Int, g)
```

*Note* - None of these functions need to be used directly and only affect PRNG implementers. Thus
regular users don't really need to worry about these redesigns and deprecations.

## Problems
### Incorrect interface

`Random` class is expected to produce uniform distribution, but:

* `Integer` has infinitely many values (bad `random`)
* `Double`/`Float` (bad `random`):
  * try to represent real values, which have an infinite range
  * floating point values have a limited range, eg. `-5.0e-324` to `5.0e-324` for `Double`
  * representable values are not equidistant from each other
  * special values `+/-Infinity`, `NaN` and `-0`
* Custom types, eg. `Uuid` or `RGB`  (bad `randomR`)

### Incorrect interface (Solution)

Correct interface is to separate `Random` class into two concepts: `Uniform` and `UniformRange`.

```haskell
class Uniform a where
  uniformM :: StatefulGen g m => g -> m a

class UniformRange a where
  uniformRM :: StatefulGen g m => (a, a) -> g -> m a
```

Taking this approach makes it possible for `Integer`, `Float` and `Double` to get
instances for `UniformRange` class, but not for `Uniform`.

In [6]:
import System.Random.Stateful
import Data.Word

data RGB a = RGB { red   :: a
                 , green :: a
                 , blue  :: a
                 } deriving (Eq, Show)

instance Uniform a => Uniform (RGB a) where
  uniformM g = RGB <$> uniformM g <*> uniformM g <*> uniformM g

In [7]:
gen <- getStdGen
fst $ uniform gen :: RGB Word8

RGB {red = 17, green = 13, blue = 125}

**Notes**:

* `StatefulGen` vs `RandomGen`: monadic vs pure
* There were suggestions of deprecating `Random` - too much breakage

```haskell
uniformM :: (Uniform a, StatefulGen g m) => g -> m a
```
```haskell
uniform :: (Uniform a, RandomGen g) => g -> (a, g)
```

## Problem
### Stateful monadic generators lack interface

* State transformer monad - when pure generator is the actual state in `StateT`

* Mutable variables - when pure generator is stored in `STRef` or an `IORef`

* Generators that depend on a large mutable state that has to be stored in some data
  structure such as a mutable vector. Here are packages that provide such generators in
  Haskell: 
  
  * [`mwc-random`](http://hackage.haskell.org/package/mwc-random),
  * [`pcg-random`](http://hackage.haskell.org/package/pcg-random),
  * [`sfmt`](https://hackage.haskell.org/package/sfmt) and
  * [`mersenne-random`](https://hackage.haskell.org/package/mersenne-random)

## State transformer monad

Passing generator around is not only inconvenient, but it also does not compose very
well. The natural solution to this problem is to use either `StateT` monad

In [8]:
uniformPhone :: RandomGen g => [AreaCode] -> g -> (Phone, g)
uniformPhone areaCodes g =
  let (i, g') = uniformR (0, length areaCodes - 1) g
      (phoneLocalNumber, g'') = uniformR (0, 9999999) g'
   in (Phone {phoneAreaCode = areaCodes !! i, phoneLocalNumber}, g'')

In [9]:
import Control.Monad.State

uniformPhoneState :: RandomGen g => [AreaCode] -> g -> (Phone, g)
uniformPhoneState areaCodes =
  runState $ do
    i <- state $ uniformR (0, length areaCodes - 1)
    phoneLocalNumber <- state $ uniformR (0, 9999999)
    pure Phone {phoneAreaCode = areaCodes !! i, phoneLocalNumber}

## New approach

Using `StateT` vs `StatefulGen` and `UniformRange`:

In [10]:
uniformPhoneStateT :: (Monad m, RandomGen g) => [AreaCode] -> StateT g m Phone
uniformPhoneStateT areaCodes = do
  i <- state $ uniformR (0, length areaCodes - 1)
  phoneLocalNumber <- state $ uniformR (0, 9999999)
  pure Phone {phoneAreaCode = areaCodes !! i, phoneLocalNumber}

In [11]:
uniformPhoneM :: StatefulGen g m => [AreaCode] -> g -> m Phone
uniformPhoneM areaCodes gen = do
  i <- uniformRM (0, length areaCodes - 1) gen
  phoneLocalNumber <- uniformRM (0, 9999999) gen
  pure Phone {phoneAreaCode = areaCodes !! i, phoneLocalNumber}

```haskell
runState :: State g a -> g -> (a, g)
runStateGen :: RandomGen g => g -> (StateGenM g -> State g a) -> (a, g)
```

In [12]:
gen <- getStdGen
runState (uniformPhoneStateT [505, 575]) gen
runStateGen gen (uniformPhoneM [505, 575])

(+1-575-828-9549,StdGen {unStdGen = SMGen 14214977679409501903 15484608899572013139})

(+1-575-828-9549,StdGen {unStdGen = SMGen 14214977679409501903 15484608899572013139})

## New approach - What's the point?

**Claim is**: this new approach works not only with pure `RandomGen`
generators, but also with the true mutable ones as well!

### Mutable variables
#### STGenM - state thread monad

For example when we are working in `ST` monad:

In [13]:
import Control.Monad.ST
import Data.STRef

uniformPhoneST :: RandomGen g => STRef s [AreaCode] -> STGenM g s -> ST s Phone
uniformPhoneST areaCodesRef stGen = do
  areaCodes <- readSTRef areaCodesRef
  uniformPhoneM areaCodes stGen

uniformPhone' :: RandomGen g => [AreaCode] -> g -> (Phone, g)
uniformPhone' areaCodes g = runSTGen g $ \stGen -> do
  areaCodesRef <- newSTRef areaCodes
  uniformPhoneST areaCodesRef stGen

#### AtomicGenM - Concurrency 

Concurrent setup cannot be done with `StateT` or `ST`:

In [14]:
import Control.Concurrent.Async (replicateConcurrently)

uniformPhones :: RandomGen g => [AreaCode] -> g -> IO ([Phone], g)
uniformPhones areaCodes g = do
  (phones, AtomicGen g') <-
    withMutableGen (AtomicGen g) $ \atomicGen -> do
      n <- uniformRM (1, 5) atomicGen
      replicateConcurrently n (uniformPhoneM areaCodes atomicGen)
  pure (phones, g')

In [15]:
print . fst =<< uniformPhones [505, 575] (mkStdGen 12)

[+1-575-255-4052,+1-575-236-8988,+1-505-962-5381]

### StatefulGen explained

`StatefulGen` looks like a monadic version of `RandomGen` class

```haskell
class Monad m => StatefulGen g m where
  {-# MINIMAL (uniformWord32|uniformWord64) #-}
  uniformWord32R :: Word32 -> g -> m Word32
  uniformWord64R :: Word64 -> g -> m Word64
  uniformWord8 :: g -> m Word8
  uniformWord16 :: g -> m Word16
  uniformWord32 :: g -> m Word32
  uniformWord64 :: g -> m Word64
  uniformShortByteString :: Int -> g -> m ShortByteString
```

### StatefulGen instances 

We already saw `StateGenM g`, `STGenM g s` and `AtomicGenM g`:

```haskell
(RandomGen g, MonadState g m) => StatefulGen (StateGenM g) m
RandomGen g                   => StatefulGen (STGenM g s) (ST s)
(RandomGen g, MonadIO m)      => StatefulGen (IOGenM g) m
(RandomGen g, MonadIO m)      => StatefulGen (AtomicGenM g) m
```

### StateGenM and MTL

Because `StateGenM` instance requires `MonadState`, it'll compose nicely with other transformers:

In [16]:
{-# LANGUAGE FlexibleContexts #-}
import Control.Monad.Reader

uniformPhoneReaderStateM
  :: (MonadReader [AreaCode] m, StatefulGen g m) => g -> m Phone
uniformPhoneReaderStateM gen = do
  areaCodes <- ask
  uniformPhoneM areaCodes gen

uniformPhoneMTL :: RandomGen g => [AreaCode] -> g -> (Phone, g)
uniformPhoneMTL = runState . runReaderT (uniformPhoneReaderStateM StateGenM)

Note that the `StateGenM` constructor, is merely a proxy type:
```haskell
data StateGenM g = StategGenM
```

### Mutable vs Immutable

A mutable generator can also have a frozen immutable counterpart:

```haskell
data StateGenM g = StategGenM
newtype StateGen g = StateGen g

newtype STGenM g s = STGenM (STRef s g)
newtype STGen g = STGen g

newtype IOGenM g = IOGenM (IORef g)
newtype IOGen g = IOGen g

newtype AtomicGenM g = AtomicGenM (IORef g)
newtype AtomicGen g = IOGen g
```

### FrozenGen

This is where `FrozenGen` comes in:

```haskell
class StatefulGen (MutableGen f m) m => FrozenGen f m where
  type MutableGen f m = (g :: Type) | g -> f
  freezeGen :: MutableGen f m -> m f
  thawGen :: f -> m (MutableGen f m)
```

#### Serialization example

Let's use `serialise` library in this example:

In [17]:
import Codec.Serialise (Serialise(..), readFileDeserialise, writeFileSerialise)

withStoredGen
  :: (Serialise f, FrozenGen f IO)
  => FilePath -- ^ File path for the seed
  -> (MutableGen f IO -> IO a) -- ^ Action that uses the mutable gen
  -> IO a
withStoredGen filepath action = do
  frozenGen <- readFileDeserialise filepath
  (result, frozenGen') <- withMutableGen frozenGen action
  writeFileSerialise filepath frozenGen'
  pure result

```haskell
withMutableGen :: FrozenGen f m => f -> (MutableGen f m -> m a) -> m (a, f)
```

### Generators that depend on a large mutable state

Finally we get to our ultimate goal. 

`mwc-random-0.15.0.1` is now capable 
of using this new interface because it provides these two instances:

```haskell
instance (s ~ PrimState m, PrimMonad m) => StatefulGen (Gen s) m where

instance PrimMonad m => FrozenGen Seed m where
  type MutableGen Seed m = Gen (PrimState m)
```

In [18]:
import System.Random.MWC (createSystemSeed)
seed <- createSystemSeed
mwc <- thawGen seed
uniformPhoneM [888] mwc

+1-888-219-6048

In [19]:
import qualified Data.Vector as VU
import System.Random.MWC (Gen, Seed, fromSeed, toSeed, createSystemSeed)

instance Serialise Seed where
  encode = encode . fromSeed
  decode = (toSeed :: VU.Vector Word32 -> Seed) <$> decode

In [20]:
seed <- createSystemSeed
writeFileSerialise "example-seed.bin" seed
:set -XTypeApplications
withStoredGen @Seed "example-seed.bin" (uniformPhoneM [505,575])

+1-505-291-1116

### Random binary data

A naive approach to generate random binary data, a.k.a. `ByteString`, is to generate a
list of `Word8`s and pack it:

In [21]:
import qualified Data.ByteString as B

randomByteStringNaive :: RandomGen g => Int -> g -> B.ByteString
randomByteStringNaive n = B.pack . take n . randoms

This approach is very inefficient for two reasons:

* In order to generate `Word8` we have to generate 64bits of random data, which means 56
  of perfectly good random bits are discarded.
* Intermediate list, if not fused will cause unnecessary allocations

### Random binary data (Solution)

Allocate a chunk of memory of a desired size and write 64bits at a time,
while making sure that the machine's CPU endianness does not affect the outcome.

In [22]:
seed <- createSystemSeed
mwcGen <- thawGen seed
B.unpack <$> uniformByteStringM 15 mwcGen

[99,85,101,84,144,148,135,179,26,189,136,76,163,87,178]

In [23]:
B.unpack $ runStateGen_ (mkStdGen 2021) (uniformByteStringM 15)

[78,232,117,189,13,237,63,84,228,82,19,36,191,5,128]

### A few words on `RandomGenM`

It works with all stateful generators that are backed by a pure PRNG.

```haskell
class (RandomGen r, StatefulGen g m) => RandomGenM g r m | g -> r where
  applyRandomGenM :: (r -> (a, r)) -> g -> m a
```

This allowed us to define monadic versions of functions from `Random` class:

```haskell
randomM :: (RandomGenM g r m, Random a) => g -> m a
randomRM :: (RandomGenM g r m, Random a) => (a, a) -> g -> m a
```

## Stats

After 5 months of work:
* [266 commits](https://github.com/idontgetoutmuch/random/compare/v1.1...v1.2-proposal)
* 150 total pull requests with a [100 of them merged](https://github.com/idontgetoutmuch/random/pulls?q=is%3Apr+is%3Amerged).
* Closed 6 existing issues and partially addressed at least 3 more.
* Exploration in API design yielded a discovery of 1 bug in GHC:
  [#18021](https://gitlab.haskell.org/ghc/ghc/-/issues/18021)

## Summary of achievements

* The quality of generated random values got much better
* Astonishing performance improvement. It only took 15 years since it was first reported
  being slow.
* Interface has been expanded dramatically
* Amount of documentation was increased quite a bit
* Modern test and benchmark suites have been added
* Very little breakage, majority of the functionality was kept backwards compatible.

## Future plans

* Generic deriving for `Uniform` and `UniformRange` classes is almost done, some of it has
  already been released
* Further improvements to floating point number generation
* `Uniform` and `UniformRange` instances for more types from base.
* Possibly another class for generating complex data structures, eg. lists, arrays, trees, etc.
* Type safety for distinguishing splittable vs non-splittable generators
* Mutable generator that works in `STM`.

## Special thanks to

* Dominic Steinitz [@idontgetoutmuch](https://github.com/idontgetoutmuch)
* Leonhard Markert [@curiousleo](https://github.com/curiousleo)
* Aleksey Khudyakov [@Shimuuar](https://github.com/Shimuuar)
* Andrew Lelechenko [@bodigrim](https://github.com/bodigrim)
* Daniel Cartwright [@chessai](https://github.com/chessai)
* Oleg Grenrus [@phadej](https://github.com/phadej)
* Richard Eisenberg [@goldfirere](https://github.com/goldfirere)
* Mathieu Boespflug [@mboes](https://github.com/mboes)

-------

# Thank You!

-------